In [ ]:
from get_data import get_data


get_data()

In [ ]:
import numpy as np
import pandas as pd


DATASET_PATH = "data/UCI HAR Dataset/"
TRAIN = "train/"
TEST = "test/"
INPUT_TYPES = [
    "body_acc_x_",
    "body_acc_y_",
    "body_acc_z_",
    "body_gyro_x_",
    "body_gyro_y_",
    "body_gyro_z_",
    "total_acc_x_",
    "total_acc_y_",
    "total_acc_z_"
]

X_train = np.dstack([
    pd.read_csv(DATASET_PATH + TRAIN + "Inertial Signals/" + signal + "train.txt", header=None, delim_whitespace=True).values for signal in INPUT_TYPES
])
X_test = np.dstack([
    pd.read_csv(DATASET_PATH + TEST + "Inertial Signals/" + signal + "test.txt", header=None, delim_whitespace=True).values for signal in INPUT_TYPES
])

y_raw_train = pd.read_csv(DATASET_PATH + TRAIN + "y_train.txt", header=None, delim_whitespace=True).values
y_raw_test = pd.read_csv(DATASET_PATH + TEST + "y_test.txt", header=None, delim_whitespace=True).values

In [ ]:
from sklearn.preprocessing import OneHotEncoder


encoder = OneHotEncoder()

y_train = encoder.fit_transform(y_raw_train).toarray()
y_test = encoder.transform(y_raw_test).toarray()

In [ ]:
from torch import Tensor
from torch.utils.data import DataLoader, TensorDataset


dataset_train = TensorDataset(Tensor(X_train), Tensor(y_train))
dataset_test = TensorDataset(Tensor(X_test), Tensor(y_test))
loader_train = DataLoader(dataset=dataset_train, batch_size=32, shuffle=True)
loader_test = DataLoader(dataset=dataset_test, batch_size=32, shuffle=True)

In [ ]:
import numpy as np


print(X_train.shape, y_train.shape, np.mean(X_train), np.std(X_train))
print(X_test.shape, y_test.shape, np.mean(X_test), np.std(X_test))

In [ ]:
from torch import nn


class Model(nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    self._lstm = nn.LSTM(input_size=9, hidden_size=32, num_layers=2, batch_first=True)
    self._linear = nn.Sequential(
      nn.Linear(32, 6)
    )

  def forward(self, input):
    outSeq, _ = self._lstm(input)
    out = outSeq[:, -1, :]
    return self._linear(out)

In [ ]:
model = Model()
print(model)

In [ ]:
import torch
from torch import nn, optim
from tqdm.notebook import tqdm


loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), weight_decay=0.0015)  

def train():
  with tqdm(desc="Batch", total=len(loader_train)) as pbatch:
    for (X, y) in loader_train:
      pred = model(X)
      loss = loss_function(pred, y)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      pbatch.set_postfix(loss=loss.item())
      pbatch.update()

def test():
  test_loss = 0.0
  accuracy = 0.0
  with torch.no_grad():
    for X, y in loader_test:
      pred = model(X)
      test_loss += loss_function(pred, y).item()
      accuracy += (torch.argmax(pred, dim=1) == torch.argmax(y, dim=1)).type(torch.float).sum()
  test_loss /= len(loader_test)
  accuracy /= len(loader_test.dataset)
  tqdm.write(f"Test -> Loss: {test_loss}, accuracy: {accuracy}")


In [ ]:
from tqdm.notebook import trange

torch.manual_seed(666)

for epoch in trange(1, 31, desc="Epoch"):
  train()
  test()

In [ ]:
test()